<h1 align="center"><img align="center" src="data/geoparse_logo.png" alt="GeoParse Logo" width="200"/></h1>
<h1 align="center">GeoParse</h1>
<h3 align="center">It's all about points <img src="data/point.png" width="10"/> lines <img src="data/line.png" width="40"/> and polygons <img src="data/polygon.png" width="30"/></h3>


#### [Website](http://geoparse.io) 
***

# Visualization

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/geoparse/geoparse/main?labpath=tutorials%2F00_visualization.ipynb)

This notebook demonstrates how to display:

* Points, lines, and polygons.
* Their heatmaps and clusters.
* Their coverage areas using geospatial cells such as H3, S2, and Geohash.
* Geospatial cells on a map based on their indexes.
* OSM roads and buildings using their IDs.
***

In [ ]:
# Add parent directory to sys.path
import os
import sys
sys.path.append(os.path.abspath(".."))

In [ ]:
import warnings

import geopandas as gpd
import h3
import pandas as pd

from geoparse.geoparse import Karta

warnings.filterwarnings("ignore")

In [ ]:
# reload all changed modules every time before executing a new line
%load_ext autoreload
%autoreload 2

In [ ]:
plp = Karta.plp

## Read from a CSV file

In [ ]:
df = pd.read_csv("data/great_britain_road_casualties-2023.csv")
df.head()

In [ ]:
len(df)

In [ ]:
plp(df)

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326") 
gdf.head()

In [ ]:
plp(gdf)

In [ ]:
plp(gdf, buffer_radius=500)

In [ ]:
plp(gdf, ring_inner_radius=1000, ring_outer_radius=1500)

In [ ]:
df['northing'] = df.latitude 
df['easting'] = df.longitude + 10
df.head()

In [ ]:
plp(df, x='easting', y='northing')

In [ ]:
plp(df, heatmap=True, cluster=True)

In [ ]:
plp(df, point_color="speed_limit", point_popup={"Speed limit": "speed_limit"})

# Read from Geospatial Files

In [ ]:
road_gdf = gpd.read_parquet("data/luxembourg_roads.parquet")
road_gdf.head()

In [ ]:
len(road_gdf)

In [ ]:
plp(road_gdf[road_gdf.highway == "motorway"], line_color='maxspeed', line_opacity=.2, line_weight=10, line_popup={'Length': 'length_m', 'Curvature':'sinuosity', 'Speed':'maxspeed'})

In [ ]:
border_gdf = gpd.read_file("data/luxembourg_border.geojson")
border_gdf

In [ ]:
plp(border_gdf)

In [ ]:
plp(border_gdf, centroid=True)

In [ ]:
plp(border_gdf, h3_res=6)

In [ ]:
plp(border_gdf, s2_res=10)

In [ ]:
plp(border_gdf, geohash_res=5)

In [ ]:
plp(border_gdf, h3_res=8, compact=True)

In [ ]:
plp(border_gdf, s2_res=14, compact=True)

In [ ]:
plp(border_gdf, geohash_res=6, compact=True)

In [ ]:
plp(border_gdf, h3_res=6, s2_res=10, geohash_res=5)

In [ ]:
plp(border_gdf, h3_res=8, s2_res=14, geohash_res=6, compact=True)

# Mutiple DataFrames

In [ ]:
plp([df, border_gdf, road_gdf[road_gdf.highway=='motorway']])

In [ ]:
plp([df, border_gdf, road_gdf[road_gdf.highway == "motorway"]], 
    point_popup={'Date': 'date', 'Max Speed': 'speed_limit'},
    poly_popup={'Name': 'name', 'Class': 'class'},
    line_color='maxspeed', line_opacity=.2, line_weight=10, 
    line_popup={'Length': 'length_m', 'Curvature':'sinuosity', 'Speed':'maxspeed'}
)

# Geospatial Cells

In [ ]:
lat, lon = 41.87, -87.78

# Get the H3 index at resolution 6 for the central point
h3_index = h3.latlng_to_cell(lat, lon, 6)

# Get adjacent H3 cells including the central cell itself
h3_cells = h3.grid_disk(h3_index, 1)  # k_ring with radius 1 returns the central cell + neighbors
h3_cells

In [ ]:
plp(cells=list(h3_cells), cell_type="h3")

# OSM Ways

In [ ]:
plp(osm_ways=[335265936])

In [ ]:
plp(osm_ways=[335265936, 53820456, 1117218957], s2_res=19, compact=True)

In [ ]:
plp(osm_ways=[893074361, 666201240])